In [1]:
from sly import Lexer
from parser import MyParser
from my_lexer import MyLexer

https://sly.readthedocs.io/en/latest/sly.html#writing-a-lexer

In [4]:
from sly import Lexer

class CalcLexer(Lexer):
    # Set of token names.   This is always required
    tokens = { ID, NUMBER, PLUS, MINUS, TIMES, #DIVIDE, 
              ASSIGN, LPAREN, RPAREN,
              IF, ELSE, WHILE,
              RESCALE, MODSW, BOOTSTRAP  # 동형암호 special function
             }

    # String containing ignored characters between tokens
    ignore = ' \t'
    
    # Other ignored patterns
    ignore_comment = r'\#.*'
    ignore_newline = r'\n+'    

    # Regular expression rules for tokens
    ID      = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUMBER  = r'\d+'
    PLUS    = r'\+'
    MINUS   = r'-'
    TIMES   = r'\*'
    #DIVIDE  = r'/'
    ASSIGN  = r'='
    LPAREN  = r'\('
    RPAREN  = r'\)'

    # Special cases
    ID['if'] = IF
    ID['else'] = ELSE
    ID['while'] = WHILE
    ID['resclae'] = RESCALE
    ID['MODSW'] = MODSW
    ID['BOOTSTRAP'] = BOOTSTRAP
   
    
    def __init__(self):
        self.lineno = 0
        self

    @_(r"(0|[1-9][0-9]*)")
    def NUMBER(self, t):
        t.value = int(t.value)
        return t
    
    @_(r'''("[^"\\]*(\\.[^"\\]*)*"|'[^'\\]*(\\.[^'\\]*)*')''')
    def STRING(self, t):
        t.value = self.remove_quotes(t.value)
        return t

    # Define a rule so we can track line numbers
    @_(r'\n+')
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    # RESCALE 동작 정의 
    
    
    # Compute column.
    #     input is the input text string
    #     token is a token instance
    def find_column(text, token):
        last_cr = text.rfind('\n', 0, token.index)
        if last_cr < 0:
            last_cr = 0
        column = (token.index - last_cr) + 1
        return column    
    
    
if __name__ == '__main__':
    data = '''x = 3 + 42
                * (s    # This is a comment
                    - t)'''
    lexer = CalcLexer()
    for tok in lexer.tokenize(data):
        print('type=%r, value=%r' % (tok.type, tok.value))

type='ID', value='x'
type='ASSIGN', value='='
type='NUMBER', value=3
type='PLUS', value='+'
type='NUMBER', value=42
type='TIMES', value='*'
type='LPAREN', value='('
type='ID', value='s'
type='MINUS', value='-'
type='ID', value='t'
type='RPAREN', value=')'
